In [1]:
import os # to check if files exist
import re #to split the text file
import random #to choose questions randomly
import requests #to send the cards to anki
from concurrent.futures import ThreadPoolExecutor #to speed up sending the cards to anki

# Read the content of the question file
file_path = "test your understanding.txt"
with open(file_path, "r") as file:
    content = file.read()

# Split the content at "Test your understanding"
substrings = content.split("Test your understanding")

# Initialize a nested list to store non-empty lines after each substring
nested_list = []

#split before new task
pattern = re.compile(r'\n(?=.{1,2}\))')

# Process each substring and split non-empty lines
for substring in substrings[1:]:  # Start from index 1 to skip the first empty substring
    lines = re.split(pattern, substring)
    nested_list.append(lines[1:]) #first line is also empty

In [82]:
# Read answer files and put them in the answer dict
answer_dict = {}

def split_tut_question_text(string):
        return re.findall(pattern= r'^(\d{1,2})\s(.{1,2}\))\s(.+)', string = string)

def append_to_answer_dict(answer_list, name):
    for line in answer_list:
        answer = split_tut_question_text(string = line)
        tut_num = int(answer[0][0]) 
        question_num = answer[0][1]
        answer_text = f"{name}- " + answer[0][2]
        if tut_num not in answer_dict:
            answer_dict[tut_num] = {question_num:answer_text}
        else: 
            if question_num not in answer_dict[tut_num]:
                answer_dict[tut_num][question_num] = answer_text
            else:
                answer_dict[tut_num][question_num] = answer_dict[tut_num][question_num] + "\n" + answer_text

answer_files = []
for file in os.listdir():
    match = re.match(r"test your understanding answers (.*)\.txt", file)
    if match:
        file_path = os.path.join(os.getcwd(), file)
        with open(file_path, "r") as file_content:
            answer_files.append([file, match.group(1), file_content.read()])

for file in answer_files:
    answers_unsplit = file[2]
    name = file[1]
    
    pattern_answer_lines = re.compile(r'\n(?=\d{1,2} .{1,2}\))')
    if(len(answers_unsplit)>1):
        lines_answers = re.split(pattern_answer_lines, answers_unsplit)
        append_to_answer_dict(lines_answers, name = name)

#function to get question num
def get_question_num(question):
    return  re.findall(pattern = r"^.{1,2}\)", string = question)[0]

#function to find answer
def find_answer(question, tut_num):
    question_num = get_question_num(question)
    answer = "Didn't answer yet"
    if tut_num in answer_dict.keys():
        if question_num in answer_dict[tut_num].keys():
            answer = answer_dict[tut_num][question_num]
    return(answer)


#function test yoursef using a random subset of numbers
def ask_question(question):
    return input(question + '\n')

def test(n = 1, ask_unanswered = True, show_all_questions_first = False, inclue_answers = True, ask_answers = True):
    """
    n: number of questions to be asked
    ask_unanswered: ask only questions without entry in answer_dict
    show_all_questions_first: print all drawn questions
    inclue_answers: include answers when printing drawn questions
    ask_answers: ask answers to questions. Inputs will be included in answer dict (which will be included in anki cards)
    """
    total_n_q = len([q for tut in nested_list for q in tut])
    if n> total_n_q:
        ValueError(f"n is too large (more than {total_n_q})") 

    question_pool = {index + 1: sublist for index, sublist in enumerate(nested_list)}

    not_answered = {}

    if ask_unanswered:
        for tut_num_1, tut in enumerate(nested_list):
            tut_num = tut_num_1 + 1 

            if tut_num not in answer_dict.keys():
                not_answered[tut_num] = tut
            else:
                not_answered[tut_num] = [q for q in tut if get_question_num(q) not in answer_dict[tut_num].keys()]
    question_pool = not_answered  

#TODO draw without replacement, make shure to have a suffiently large question pool also if answered all questions
    questions = []
    for _ in range(n): #draw n random samples
        #get random tut
        tut_num = random.choice(list(question_pool.keys()))
        # choose a random question from the tut
        question = random.choice(question_pool[tut_num])

        if show_all_questions_first:
            print(f"Tut: {tut_num}\n{question}")
            if inclue_answers:
                print(f"Answer: {find_answer(question, tut_num)}")
        questions.append(f"{tut_num} {question}")
           
    if ask_answers:
        name = input("Under which name do you want to answer the questions?" + "\n")
        if len(name) < 1:
            name = "x"
        # List to store answers
        answers = []

        # Ask each question and store the answers
        for question in questions:
            tut_question_text = split_tut_question_text(question)
            answer = ask_question(question)
            if len(answer)>0:
                print(question)
                print(answer)
                answer = f"{tut_question_text[0][0]} {tut_question_text[0][1]} {answer}"
                answers.append(answer)


        # Convert the list of answers to lines in a .txt document
        filename = f"test your understanding answers {name}.txt"

        # Check if the file exists
        if os.path.exists(filename):
            # Append content to an existing file
            with open(filename, "a") as file:
                for answer in answers:
                    file.write(f"{answer}\n")
        else:
            # Create a new file
            with open(filename, "w") as file:
                for answer in answers:
                    file.write(f"{answer}\n")

        #put the answers into the answer_dict            
        append_to_answer_dict(answers, name)

In [83]:
test(n = 2) #ask questions

8 b) Describe and compare the three missing data mechanisms. Give an exam-
ple of each mechanism.
MCAR: missing completely at random means there is no systematic reason why data is missng. E.g it was not measured or not measured for a random subset of the Sample. The data MAR: That data is mssing depends on some feature which was measured. This can be modeled and explained MNAR: The data is missing dependent on unobsered features. This is not ignorable.. 
8 e) Describe the E step and M step formally based on your answer to the
previous question. What can we say about the EM algorithm after each
iteration of the E and M step? And asymptotically?
E step: compute missing data distribution usign current parameter estimates in order to 'compute' the expected complete data distribution (Q-Function). In the M step we take the argmax of \theta for the Q-Function to get the parameter estimate for the next E step.


In [ ]:
#TODO make shure Anki is installed and currently running on the machine
#TODO make shure anki-connect is installed https://github.com/FooSoft/anki-connect
#answer_dict = {} #TODO if you don' want to include any answers, just uncomment this line 
deck_name = "ML 2023 Test your understanding"  #TODO Change this to your deck name (this deck will be created if it doesn't exist allready)
anki_url = "http://localhost:8765"  # AnkiConnect default URL

#put all questions and Answers (if exist) into Anki list
anki_list = [{"note_type": "Basic",
               "fields": {"Front":f"Tut {tut_i+1}\n{q}",
                          "Back": find_answer(q, tut_i+1)},
                "tags": [f"Tut_{tut_i+1}"]} for tut_i, tut in enumerate(nested_list) for q in tut]

#check if deck exists, create if not
def deck_exists(deck_name):
    # Build the AnkiConnect API request payload for deck check
    payload = {
        "action": "deckNames",
        "version": 6,
    }

    # Make the API request to AnkiConnect to get the list of existing deck names
    response = requests.post(anki_url, json=payload)

    # Check the response status
    if response.status_code == 200:
        existing_decks = response.json()["result"]
        return deck_name in existing_decks
    else:
        print(f"An error occurred while checking for existing decks: {response.text}")
        return False
def create_deck(deck_name):
    # Build the AnkiConnect API request payload for creating a deck
    payload = {
        "action": "createDeck",
        "version": 6,
        "params": {
            "deck": deck_name,
        }
    }

    # Make the API request to AnkiConnect to create the deck
    response = requests.post(anki_url, json=payload)

    # Check the response status
    if response.status_code == 200:
        print(f"Deck '{deck_name}' created successfully.")
    else:
        print(f"An error occurred while creating the deck: {response.text}")


if not deck_exists(deck_name):
    # Create the deck if it doesn't exist
    create_deck(deck_name)
else:
    print(f"Deck '{deck_name}' already exists.")
    if "Y" != input("Type 'Y' if you want to create new cards in deck: " + deck_name):
        raise ValueError("Didn't type 'Y'")

#create cards
def create_anki_card(payload):
    response = requests.post(anki_url, json=payload)
    return response.json()

def create_anki_cards(cards_data):
    with ThreadPoolExecutor() as executor:
        payloads = []
        for card in cards_data:
            tags = ["ML", "Data_Science"]
            note_type = card["note_type"]
            fields = card["fields"]
            tags.extend(card["tags"])

            # Construct the payload for creating a new note
            payload = {
                "action": "addNote",
                "version": 6,
                "params": {
                    "note": {
                        "deckName": deck_name,
                        "modelName": note_type,
                        "fields": fields,
                        "tags": tags,  # Add tags if needed
                    }
                }
            }
            payloads.append(payload)
        # Use ThreadPoolExecutor to parallelize the requests
        responses = list(executor.map(create_anki_card, payloads))

        # Print the responses (optional)
        for response in responses:
            print(response)

create_anki_cards(anki_list) #Took 35sec on my machine